In [1]:
!pip install torch
!pip install torch_geometric
# !pip install scipy
!pip install numpy

  Using cached torch-2.4.1-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

In [28]:
class GCNModel(nn.Module):
    def __init__(self, in_feature_len, hidden_feature_len, output_feature_len):
        super(GCNModel, self).__init__()
        self.layer1 = GCNConv(in_feature_len, hidden_feature_len)
        self.layer2 = GCNConv(hidden_feature_len, output_feature_len)
        self.relu = F.relu
        self.dropout = F.dropout

    def forward(self, x, adj_matrix):
        # First GCN layer
        x = self.layer1(x, adj_matrix)
        x = self.relu(x)
        x = self.dropout(x, p=0.5)
        # Second GCN layer
        x = self.layer2(x, adj_matrix)
        return F.log_softmax(x, dim=1)



In [29]:

dataset = Planetoid(root='data/Planetoid', name='Cora')

In [30]:
input_features = dataset.num_node_features
num_classes = dataset.num_classes
print(dataset[0], input_features, num_classes)
model = GCNModel(input_features, 16, num_classes)


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708]) 1433 7


In [33]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
data = dataset[0]

def train_model():
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(200):
    loss_value = train_model()
    print(f'Epoch: {epoch+1:03d}, Loss: {loss_value:.4f}')

Epoch: 001, Loss: 0.0135
Epoch: 002, Loss: 0.0271
Epoch: 003, Loss: 0.0249
Epoch: 004, Loss: 0.0230
Epoch: 005, Loss: 0.0356
Epoch: 006, Loss: 0.0241
Epoch: 007, Loss: 0.0278
Epoch: 008, Loss: 0.0306
Epoch: 009, Loss: 0.0169
Epoch: 010, Loss: 0.0126
Epoch: 011, Loss: 0.0246
Epoch: 012, Loss: 0.0178
Epoch: 013, Loss: 0.0304
Epoch: 014, Loss: 0.0180
Epoch: 015, Loss: 0.0300
Epoch: 016, Loss: 0.0190
Epoch: 017, Loss: 0.0294
Epoch: 018, Loss: 0.0198
Epoch: 019, Loss: 0.0153
Epoch: 020, Loss: 0.0116
Epoch: 021, Loss: 0.0228
Epoch: 022, Loss: 0.0076
Epoch: 023, Loss: 0.0139
Epoch: 024, Loss: 0.0220
Epoch: 025, Loss: 0.0331
Epoch: 026, Loss: 0.0219
Epoch: 027, Loss: 0.0272
Epoch: 028, Loss: 0.0193
Epoch: 029, Loss: 0.0105
Epoch: 030, Loss: 0.0186
Epoch: 031, Loss: 0.0223
Epoch: 032, Loss: 0.0208
Epoch: 033, Loss: 0.0122
Epoch: 034, Loss: 0.0189
Epoch: 035, Loss: 0.0279
Epoch: 036, Loss: 0.0143
Epoch: 037, Loss: 0.0250
Epoch: 038, Loss: 0.0105
Epoch: 039, Loss: 0.0282
Epoch: 040, Loss: 0.0161


In [43]:
# Evaluation

def eval():
    model.eval()
    with torch.no_grad():
        output = model(data.x, data.edge_index).argmax(dim=1)
        all_correct = (output[data.test_mask]== data.y[data.test_mask]).sum()
    return (all_correct/(len(data.y[data.test_mask]))).item()

print(f"Test accuracy is:{eval():.4f}")


Test accuracy is:0.7560
